In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## BERT+2CNN-1D Model with max length 64

## Importing the Dataset and Tokenization
The pre_process_dataset(values) and data_process(data,labels) methods have been implemented by [@ZeroxTM](https://github.com/ZeroxTM) in [this repository](https://github.com/ZeroxTM/BERT-CNN-Fine-Tuning-For-Hate-Speech-Detection-in-Online-Social-Media) the methods are in Pre_Process.py.

In [ ]:
import re
import string
import pandas as pd
import emoji
from transformers import BertTokenizer
import numpy as np

DATASET_PATH = '/content/drive/MyDrive/NLP-project/dataset/train.csv'

def balance_dataset(dataset):
    min_len = (dataset['label']==1).sum()
    sampled_from_0s = dataset[dataset['label']==0].sample(n = 2*min_len, random_state=42)
    return pd.concat([dataset[dataset['label']==1], sampled_from_0s])

def read_dataset(path):
    dataset = pd.read_csv(path)
    dataset = dataset[~pd.isna(dataset['comment_text'])]
    dataset['label'] = dataset.values[:,2:].sum(axis=1) > 0
    dataset['label'] = dataset['label'].astype(int)
    balanced_dataset = balance_dataset(dataset)
    return balanced_dataset['comment_text'].tolist(), balanced_dataset['label']

def data_process(data, labels):
    input_ids = []
    attention_masks = []
    max_length = 64
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    for sentence in data:
        bert_inp = bert_tokenizer.__call__(sentence, max_length=max_length,
                                           padding='max_length', pad_to_max_length=True,
                                           truncation=True, return_token_type_ids=False)

        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])
    input_ids = np.asarray(input_ids)
    attention_masks = np.array(attention_masks)
    labels = np.array(labels)

    return input_ids, attention_masks, labels

def pre_process_dataset(values):
    new_values = list()
    # Emoticons
    emoticons = [':-)', ':)', '(:', '(-:', ':))', '((:', ':-D', ':D', 'X-D', 'XD', 'xD', 'xD', '<3', '</3', ':\*',
                 ';-)',
                 ';)', ';-D', ';D', '(;', '(-;', ':-(', ':(', '(:', '(-:', ':,(', ':\'(', ':"(', ':((', ':D', '=D',
                 '=)',
                 '(=', '=(', ')=', '=-O', 'O-=', ':o', 'o:', 'O:', 'O:', ':-o', 'o-:', ':P', ':p', ':S', ':s', ':@',
                 ':>',
                 ':<', '^_^', '^.^', '>.>', 'T_T', 'T-T', '-.-', '*.*', '~.~', ':*', ':-*', 'xP', 'XP', 'XP', 'Xp',
                 ':-|',
                 ':->', ':-<', '$_$', '8-)', ':-P', ':-p', '=P', '=p', ':*)', '*-*', 'B-)', 'O.o', 'X-(', ')-X']

    for value in values:
        # Remove dots
        text = value.replace(".", "").lower()
        text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
        users = re.findall("[@]\w+", text)
        for user in users:
            text = text.replace(user, "<user>")
        urls = re.findall(r'(https?://[^\s]+)', text)
        if len(urls) != 0:
            for url in urls:
                text = text.replace(url, "<url >")
        for emo in text:
            if emo in emoji.UNICODE_EMOJI:
                text = text.replace(emo, "<emoticon >")
        for emo in emoticons:
            text = text.replace(emo, "<emoticon >")
        numbers = re.findall('[0-9]+', text)
        for number in numbers:
            text = text.replace(number, "<number >")
        text = text.replace('#', "<hashtag >")
        text = re.sub(r"([?.!,¿])", r" ", text)
        text = "".join(l for l in text if l not in string.punctuation)
        text = re.sub(r'[" "]+', " ", text)
        new_values.append(text)
    return new_values

def load_and_process(path):
    data, labels = read_dataset(path)
    num_of_labels = len(labels.unique())
    input_ids, attention_masks, labels = data_process(pre_process_dataset(data), labels)

    return input_ids, attention_masks, labels


## Training Phase

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import gc
from preprocessing import load_and_process
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from models import BERT_2CNN_1D

def train():
    model.train()
    total_loss = 0
    total = len(train_dataloader)

    for i, batch in enumerate(train_dataloader):
        step = i+1
        percent = "{0:.3f}".format(100 * (step / float(total)))
        lossp = "{0:.3f}".format(total_loss/step)
        filledLength = int(100 * step // total)
        bar = '█' * filledLength + '>'  *(filledLength < 100) + '.' * (99 - filledLength)
        print(f'\rBatch {step}/{total} |{bar}| {percent}% complete, loss={lossp}', end='')
        batch = [r.to(device) for r in batch]
        sent_id, mask, labels = batch
        del batch
        gc.collect()
        torch.cuda.empty_cache()
        model.zero_grad()
        preds = model(sent_id.to(device).long(), mask)
        loss = cross_entropy(preds, labels)
        total_loss += float(loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    gc.collect()
    torch.cuda.empty_cache()

    avg_loss = total_loss / len(train_dataloader)
    
    return avg_loss


# function for evaluating the model
def evaluate():
    print("\n\nEvaluating...")
    model.eval()
    total_loss= 0
    total = len(val_dataloader)
    for i, batch in enumerate(val_dataloader):
        step = i+1
        percent = "{0:.3f}".format(100 * (step / float(total)))
        lossp = "{0:.3f}".format(total_loss/step)
        filledLength = int(100 * step // total)
        bar = '█' * filledLength + '>' * (filledLength < 100) + '.' * (99 - filledLength)
        print(f'\rBatch {step}/{total} |{bar}| {percent}% complete, loss={lossp}', end='')
        # push the batch to gpu
        batch = [t.to(device) for t in batch]
        sent_id, mask, labels = batch
        del batch
        gc.collect()
        torch.cuda.empty_cache()
        with torch.no_grad():
            preds = model(sent_id, mask)
            loss = cross_entropy(preds, labels)
            total_loss += float(loss.item())

    gc.collect()
    torch.cuda.empty_cache()
    val_avg_loss = total_loss / len(val_dataloader)

    return val_avg_loss

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the dataset
input_ids, attention_masks, labels = load_and_process(DATASET_PATH)
df = pd.DataFrame(list(zip(input_ids, attention_masks)), columns=['input_ids', 'attention_masks'])

train_text, temp_text, train_labels, temp_labels = train_test_split(df, labels,
                             random_state=42, test_size=0.2, stratify=labels)

val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
                         random_state=42, test_size=0.5, stratify=temp_labels)

del temp_text
gc.collect()
torch.cuda.empty_cache()

train_count = len(train_labels)
test_count = len(test_labels)
val_count = len(val_labels)


# for train set
train_seq = torch.tensor(train_text['input_ids'].tolist())
train_mask = torch.tensor(train_text['attention_masks'].tolist())
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(val_text['input_ids'].tolist())
val_mask = torch.tensor(val_text['attention_masks'].tolist())
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(test_text['input_ids'].tolist())
test_mask = torch.tensor(test_text['attention_masks'].tolist())
test_y = torch.tensor(test_labels.tolist())

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

model = BERT_2CNN_1D()

model = model.to(device)

# Adam optimizer
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

cross_entropy = nn.NLLLoss()

epochs = 3
current = 1
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train()

    # evaluate model
    valid_loss = evaluate()
    
    print(f'\n\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

    current = current + 1


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]


Epoch 1 / 3:
Batch 1217/1217 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.006

Evaluating...
Batch 153/153 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.006

Training Loss: 0.006
Validation Loss: 0.006

Epoch 2 / 3:
Batch 1217/1217 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.004

Evaluating...
Batch 153/153 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.006

Training Loss: 0.004
Validation Loss: 0.006

Epoch 3 / 3:
Batch 1217/1217 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.002

Evaluating...
Batch 153/153 |███████████████████████████████████████████████████████████████████

## Evaluation(only on 3000 sequences)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

with torch.no_grad():
    gc.collect()
    torch.cuda.empty_cache()
    preds = model(test_seq[:3000].to(device), test_mask[:3000].to(device))
    preds = preds.detach().cpu().numpy()


print("Performance:")
# model's performance
preds = np.argmax(preds, axis=1)
print('Classification Report')
print(classification_report(test_y[:3000], preds))

Performance:
Classification Report
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1989
           1       0.92      0.87      0.90      1011

    accuracy                           0.93      3000
   macro avg       0.93      0.92      0.92      3000
weighted avg       0.93      0.93      0.93      3000



## Evaluation on the whole of test set
The model needs about 3 GiB memory to evaluate the test set, but because of bad memory fragmentation, Pytorch cannot allocate enough memory. So, I partitioned the test set into segments of size at most 1000 sequences to evaluate the model. I compute the confusion matrix in the first cell and precision, recall, f1-score, and accuracy in the second cell.

In [ ]:
from sklearn.metrics import confusion_matrix

gc.collect()
torch.cuda.empty_cache()

conf_mat = np.zeros((2,2))
for i in range(0,len(test_seq),1000):
  with torch.no_grad():
      gc.collect()
      torch.cuda.empty_cache()
      preds = model(test_seq[i:i+1000].to(device), test_mask[i:i+1000].to(device))
      preds = preds.detach().cpu().numpy()
      preds = np.argmax(preds, axis=1)
  conf_mat += confusion_matrix(test_y[i:i+1000],preds)

In [ ]:
conf_mat

array([[3133.,  112.],
       [ 203., 1420.]])

In [ ]:
p_0 = np.sum(conf_mat[0,0]/np.sum(conf_mat[:,0]))
p_1 = np.sum(conf_mat[1,1]/np.sum(conf_mat[:,1]))
r_0 = np.sum(conf_mat[0,0]/np.sum(conf_mat[0,:]))
r_1 = np.sum(conf_mat[1,1]/np.sum(conf_mat[1,:]))
f1_0 = 2*(p_0*r_0/(p_0+r_0))
f1_1 = 2*(p_1*r_1/(p_1+r_1))

macro_precision = (p_0 + p_1)/2
macro_recall = (r_0 + r_1)/2
macro_f1 = (f1_0 + f1_1)/2
accuracy = np.sum(np.diag(conf_mat))/np.sum(conf_mat)

print(f'Macro precision: {"{0:0.2f}".format(macro_precision)},\n\
Macro recall: {"{0:0.2f}".format(macro_recall)},\n\
Macro f1: {"{0:0.2f}".format(macro_f1)},\n\
Accuracy: {"{0:0.2f}".format(accuracy)}')

Macro precision: 0.93,
Macro recall: 0.92,
Macro f1: 0.93,
Accuracy: 0.94


## Saving the model

In [ ]:
torch.save(model, 'toxic_comment_classifier_BERT_2CNN_1D(max_len_64).pt')